In [2]:
deployment = "gpt4"
model = "gpt-4"

# 回顾

In [29]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("OPENAI_API_BASE")
    )

import openai
def analyze_user_review(text):
    messages = []
    messages.append( {"role": "system", 
                      "content": """
                      You are an assistant. 
                      Please, analyze the user reviews according to the following instruction: 
                      If the review is postive, you should output 'Y', otherwise output 'N'
                      """})
    messages.append( {"role": "user", "content": text})
    response = client.chat.completions.create(
        model=deployment, 
        messages=messages,
        temperature=0.5,
        max_tokens = 100
    )
    return response.choices[0].message.content

In [30]:
analyze_user_review("服务热情周到。")

'Y'

In [8]:
analyze_user_review("不理不睬。")

'N'

In [9]:
analyze_user_review("用餐环境差，等待时间长。")

'N'

In [12]:
!pip install openai==1.3.6

In [15]:
!pip install langchain==0.0.343

# 提示词模版

In [3]:
from langchain import PromptTemplate, OpenAI, LLMChain
import os
from langchain.chat_models import AzureChatOpenAI
# from langchain.chat_models import ChatOpenAI #直接访问OpenAI的GPT服务

prompt_template = "What is a good name for a company that makes {product}? And only return the best one."
#llm = ChatOpenAI(model_name="gpt-4", temperature=0) #直接访问OpenAI的GPT服务
llm = AzureChatOpenAI(model = deployment,
                      temperature=0, max_tokens=200)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain.run("colorful socks")

/Users/mbj0593/anaconda3/lib/python3.11/site-packages/langchain/chat_models/azure_openai.py:156: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://superman-ai.openai.azure.com/ to https://superman-ai.openai.azure.com/openai.
  warnings.warn(


'Sock Spectrum'

In [4]:
products = [{"product":"'cloudnative devops platform'"},
            {"product":"'Noise cancellation headphone'"}, 
            {"product":"colorful socks"}]
llm_chain.apply(products)

[{'text': 'SkyOpsForge'},
 {'text': 'Silent Symphony'},
 {'text': 'Sock Spectrum'}]

# API调用链
## HTTP request chain

In [5]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chains import LLMRequestsChain
from langchain.chat_models import AzureChatOpenAI
# from langchain.chat_models import ChatOpenAI #直接访问OpenAI的GPT服务

#llm = ChatOpenAI(model_name="gpt-4", temperature=0) #直接访问OpenAI的GPT服务
llm = AzureChatOpenAI(model = deployment, temperature=0, max_tokens=200)
def query_baidu(question):
      template = """Between >>> and <<< are the raw search result text from web.
      Extract the answer to the question '{query}' or say "not found" if the information is not contained.
      Use the format
      Extracted:<answer or "not found">
      >>> {requests_result} <<<
      Extracted:"""

      PROMPT = PromptTemplate(
          input_variables=["query", "requests_result"],
          template=template,
      )

      inputs = {
          "query": question,
          "url": "http://www.baidu.com/s?wd=" + question.replace(" ", "+")
      }
      requests_chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=PROMPT), output_key="query_info", verbose=True)
      res = requests_chain.run(inputs)
      return res

/Users/mbj0593/anaconda3/lib/python3.11/site-packages/langchain/chat_models/azure_openai.py:156: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://superman-ai.openai.azure.com/ to https://superman-ai.openai.azure.com/openai.
  warnings.warn(


In [6]:
query_baidu("今天北京天气？")



> Entering new LLMRequestsChain chain...

> Finished chain.


'今天北京天气晴，东北风1级，气温-6~5°C，空气质量指数27 优。'

In [23]:
# import os
# os.environ["SERPER_API_KEY"] = ""
# https://serper.dev
from langchain.utilities import GoogleSerperAPIWrapper
def query_web(question):
    search = GoogleSerperAPIWrapper()
    return search.run(question)


In [24]:
query_web("今天北京天气？")

'35°F'

# 调用链

In [26]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import SequentialChain
from langchain.chat_models import AzureChatOpenAI
# from langchain.chat_models import ChatOpenAI #直接访问OpenAI的GPT服务

#llm = ChatOpenAI(model_name="gpt-4", temperature=0) #直接访问OpenAI的GPT服务
llm = AzureChatOpenAI(model = deployment, temperature=0, max_tokens=200)

summarizing_prompt_template = """
Summarize the following content into a sentence less than 20 words:
---
{content}

"""
summarizing_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(summarizing_prompt_template),
    output_key = "summary"
)

translating_prompt_template = """
translate "{summary}" into Chinese:

"""

translating_chain = LLMChain(
    llm = llm,
    prompt=PromptTemplate.from_template(translating_prompt_template),
    output_key = "translated"
)

overall_chain = SequentialChain(
    chains=[summarizing_chain, translating_chain],
    input_variables=["content"],
    output_variables=[ "summary","translated"],
    verbose=True
)


/Users/mbj0593/anaconda3/lib/python3.11/site-packages/langchain/chat_models/azure_openai.py:156: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://superman-ai.openai.azure.com/ to https://superman-ai.openai.azure.com/openai.
  warnings.warn(


In [27]:
res = overall_chain("""
LangChain is a framework for developing applications powered by language models. It enables applications that are:

Data-aware: connect a language model to other sources of data
Agentic: allow a language model to interact with its environment
The main value props of LangChain are:

Components: abstractions for working with language models, along with a collection of implementations for each abstraction. Components are modular and easy-to-use, whether you are using the rest of the LangChain framework or not
Off-the-shelf chains: a structured assembly of components for accomplishing specific higher-level tasks
Off-the-shelf chains make it easy to get started. For more complex applications and nuanced use-cases, components make it easy to customize existing chains or build new ones.
""")

print("summary:"+res["summary"])

print("中文:"+res["translated"])



> Entering new SequentialChain chain...

> Finished chain.
summary:LangChain is a modular framework for creating data-aware and agentic applications using language models with ready-to-use components and chains.
中文:"LangChain 是一个模块化框架，用于使用语言模型创建数据感知和具有代理性的应用程序，提供现成的组件和链条。"
